In [1]:
# Import modules

import requests
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from env import client_id, client_secret, redirect_uri

In [2]:
# Define Spotify environment
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret
os.environ['SPOTIPY_REDIRECT_URI'] = redirect_uri

In [3]:
# What I'm looking for
scope = 'user-top-read'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# Retrieve 30 Day Top
short_term_tracks = sp.current_user_top_tracks(limit=50,offset=0,time_range='short_term')
# Write to JSON
with open('short_term_top_tracks.json','w',encoding='utf-8') as f:
    json.dump(short_term_tracks,f, ensure_ascii = False, indent = 4)

# Retrieve 6 Month Top
medium_term_tracks = sp.current_user_top_tracks(limit=50, offset = 0, time_range = 'medium_term')
# Write to JSON
with open('medium_term_top_tracks.json','w',encoding='utf-8') as f:
    json.dump(medium_term_tracks,f, ensure_ascii = False, indent = 4)

# Retrieve Years Top
long_term_tracks = sp.current_user_top_tracks(limit=50, offset = 0, time_range = 'long_term')
# Write to JSON
with open('long_term_top_tracks.json','w',encoding='utf-8') as f:
    json.dump(long_term_tracks,f, ensure_ascii = False, indent = 4)

In [4]:
# Open all JSON
with open('short_term_top_tracks.json') as f:
    short_data = json.load(f)
with open('medium_term_top_tracks.json') as f:
    medium_data = json.load(f)
with open('long_term_top_tracks.json') as f:
    long_data = json.load(f)

### Create dataframes for each term of data

In [5]:
artist_name = []
song_name = []
song_uri = []
album_name = []
for track in short_data['items']:
    track['album']
    artist_name.append(track['artists'][0]['name'])
    song_name.append(track['name'])
    song_uri.append(track['uri'])
    album_name.append(track['album']['name'])

short_df = pd.DataFrame({'artist':artist_name,
'song':song_name,
'album':album_name,
'song_uri':song_uri,
'term':'short'})

In [6]:
artist_name = []
song_name = []
song_uri = []
album_name = []
for track in medium_data['items']:
    track['album']
    artist_name.append(track['artists'][0]['name'])
    song_name.append(track['name'])
    song_uri.append(track['uri'])
    album_name.append(track['album']['name'])

medium_df = pd.DataFrame({'artist':artist_name,
'song':song_name,
'album':album_name,
'song_uri':song_uri,
'term':'medium'})

In [7]:
artist_name = []
song_name = []
song_uri = []
album_name = []
for track in long_data['items']:
    track['album']
    artist_name.append(track['artists'][0]['name'])
    song_name.append(track['name'])
    song_uri.append(track['uri'])
    album_name.append(track['album']['name'])

long_df = pd.DataFrame({'artist':artist_name,
'song':song_name,
'album':album_name,
'song_uri':song_uri,
'term':'long'})

In [8]:
print(short_df.shape)
print(medium_df.shape)
print(long_df.shape)

(50, 5)
(50, 5)
(50, 5)


In [9]:
# Combine all three dataframes into one top songs dataframe
dfs = [short_df, medium_df, long_df]
df = pd.concat(dfs)

In [10]:
df.shape

(150, 5)

In [11]:
df.shape

(150, 5)

In [12]:
first_100 = pd.DataFrame(sp.audio_features(tracks = df.song_uri.head(100)))

In [13]:
first_100.shape

(100, 18)

In [14]:
last_50 = pd.DataFrame(sp.audio_features(tracks= df.song_uri.tail(50)))

In [15]:
last_50.shape

(50, 18)

In [16]:
top_100_df = pd.merge(df,first_100,how='right',left_on='song_uri',right_on='uri')
top_50_df = pd.merge(df,last_50,how='right',left_on='song_uri',right_on='uri')

In [17]:
df = pd.concat([top_100_df, top_50_df])

In [18]:
df.shape

(200, 23)

In [19]:
df.drop_duplicates(inplace=True)

In [24]:
df.drop(columns = ['analysis_url','track_href','uri','id','type','song_uri'],inplace=True)

In [29]:
df[df.term == 'short']

,artist,song,album,term,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,King Crimson,Discipline,Discipline,short,0.750,0.7220,9,-11.579,1,0.0346,0.683000,0.804000,0.0704,0.8870,106.803,313000,3
1,"Tomorrow, St. Peter",Figurine,"Tomorrow, St. Peter",short,0.492,0.8310,7,-5.367,1,0.0430,0.269000,0.000054,0.1620,0.4800,139.903,263576,3
2,Incubus,Aqueous Transmission,Morning View,short,0.415,0.4560,9,-12.082,1,0.0352,0.005010,0.763000,0.1780,0.0935,151.937,466560,4
3,B'z,Into Free -Dangan-,B'z,short,0.386,0.8860,8,-6.219,1,0.0920,0.020600,0.000006,0.2490,0.1260,135.050,281210,4
4,Totorro,Chevalier Bulltoe,Home Alone,short,0.559,0.7180,2,-7.062,1,0.0715,0.246000,0.750000,0.1140,0.2030,138.013,212016,4
5,Delta Sleep,The Softest Touch,Spring Island,short,0.375,0.7070,11,-7.069,0,0.0331,0.046700,0.261000,0.1430,0.2640,133.682,329636,4
7,King Crimson,Indiscipline,Discipline,short,0.355,0.4520,7,-10.771,0,0.0516,0.017300,0.310000,0.0941,0.5850,122.274,273000,4
8,Rooney,Are You Afraid?,Calling The World,short,0.565,0.7870,4,-4.958,0,0.0479,0.010300,0.000000,0.1330,0.3370,130.994,253987,4
9,Bay Faction,It's Perfect,Florida Guilt,short,0.621,0.8260,8,-7.507,1,0.0556,0.089500,0.000018,0.0734,0.9630,160.003,122679,4
11,Delta Sleep,The Detail,Spring Island,short,0.477,0.6370,7,-7.244,1,0.0385,0.056000,0.014300,0.0904,0.5450,112.574,269174,4
